# 1. Set Up

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid

In [2]:
import matplotlib.pyplot as plt

In [3]:
from tqdm.auto import tqdm

## 1.1 Torch Check

In [4]:
torch.__version__

'2.0.1'

In [5]:
torch.cuda.is_available()

True

## 1.2 Setting The Environment

In [6]:
torch.manual_seed(42)

# 2. Utility Functions

In [7]:
def get_generator_block(input_dim, output_dim):
    g_block = nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )
    return g_block

In [8]:
# Unit Test Forgenerator Block
def test_gen_block(in_features, out_features, num_test=10):
    block = get_generator_block(in_features, out_features)

    # Check the three parts
    assert len(block) == 3
    assert type(block[0]) == nn.Linear
    assert type(block[1]) == nn.BatchNorm1d
    assert type(block[2]) == nn.ReLU
    
    # Check the output shape
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    print(test_output.std())
    assert tuple(test_output.shape) == (num_test, out_features)
    assert test_output.std() > 0.55
    assert test_output.std() < 0.65

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

tensor(0.5600, grad_fn=<StdBackward0>)
tensor(0.6033, grad_fn=<StdBackward0>)
Success!


## 2.2 Noise

In [9]:
def get_noise(n_samples, z_dim, device ='cuda'):
    """
    n_samples : no. of samples to generate
    z_dim : Dimension of the noise vector
    device : cpuu or cuda, where to generate
    """
    
    return torch.randn(n_samples, z_dim).to(device)

In [10]:
# Verify the noise vector function
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    
#     print(noise)
#     print(noise.std())
#     print(torch.abs(noise.std() - torch.tensor(1.0)))
    # Make sure a normal distribution was used
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device).startswith(device)

test_get_noise(1000, 100, 'cpu')
if torch.cuda.is_available():
    test_get_noise(1000, 32, 'cuda')
print("Success!")

Success!


## 2.3 Discriminator

In [15]:
def get_discriminator_block(input_dim, output_dim):
    block = nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)
    )
    return block

In [16]:
# Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5

test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


# 3. Utility Functions

In [11]:
class Generator(nn.Module):
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        """
        z_dim = dimension of the noise vector
        im_dim = dimnsion of the image(For MNIST 28*28 = 784)
        hidden_dim = No. of Units in the One Hidden Layer
        """
        super(Generator, self).__init__()
        
        #Build The Neural Network
        self.gen = nn.Sequential(
            #Generator Painting an Image
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            #Making that Image into The Actual Image Dimensions
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
        
    def forward(self, noise):
        """
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        """
        return self.gen(noise)
    
    def get_gen(self):
        """
        Return The Model
        """
        return self.gen

In [12]:
x = Generator(5, 10, 20)

In [13]:
x.get_gen()

Sequential(
  (0): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (1): Sequential(
    (0): Linear(in_features=20, out_features=40, bias=True)
    (1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (2): Sequential(
    (0): Linear(in_features=40, out_features=80, bias=True)
    (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (3): Sequential(
    (0): Linear(in_features=80, out_features=160, bias=True)
    (1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (4): Linear(in_features=160, out_features=10, bias=True)
  (5): Sigmoid()
)

In [21]:
class Discriminator(nn.Module):
    def __init__(self, im_dim=784, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim *2, hidden_dim),
            #Transform the final output into a single value
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, image):
        return self.disc(image)
    
    def get_disc(self):
        return self.disc

In [22]:
y = Discriminator(5, 10)


In [23]:
y.get_disc()

Sequential(
  (0): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (1): Sequential(
    (0): Linear(in_features=40, out_features=20, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (2): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (3): Linear(in_features=10, out_features=1, bias=True)
)